In [14]:
import json

with open('vocab.json', 'r', encoding='utf-8') as f:
    vocab = json.load(f)

In [15]:
len(vocab)
vocab['попа']

19150

In [16]:
from datasets import load_from_disk
load_dataset = load_from_disk("dataset")
len(load_dataset)


Loading dataset from disk:   0%|          | 0/112 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
load_dataset.shard(4, index=1)[1]

{'tokens': ['люблю',
  'люблю',
  'боли',
  'горле',
  'понимания',
  'важен',
  'каждый',
  'миг',
  'пусть',
  'радости',
  'пусть',
  'слёзы',
  'невзгоды',
  'люблю',
  'люблю',
  'дрожи',
  'коленях',
  'гусиной',
  'кожи',
  'порой',
  'вызываешь',
  'веришь',
  'люблю',
  'люблю',
  'дорогой',
  'скажу',
  'тебе',
  'смогу',
  'продолжить',
  'жить',
  'раньше',
  'поверь',
  'смогу',
  'сберегу',
  'мгновенья',
  'фальши',
  'смогу',
  'предрассудков',
  'уверена',
  'смогу',
  'говорить',
  'боли',
  'горле',
  'буду',
  'люблю']}

In [ ]:
from tqdm import tqdm
import random
total_pairs = 0
short = 0
w = 0.5
N = 10
idx = random.sample(range(len(load_dataset)), N)

pairs = 0
for i in tqdm(idx):
    L = len(load_dataset[i]["tokens"])
    if L > 2*w:
        pairs += (L - 2*w) * (2*w)

avg_pairs = pairs / N
est_total = int(avg_pairs * len(load_dataset))

print("avg_pairs_per_doc:", avg_pairs)
print("estimated_total_pairs:", est_total)

100%|██████████| 100000/100000 [00:39<00:00, 2527.61it/s]

avg_pairs_per_doc: 290.94415
estimated_total_pairs: 3308929347


In [62]:
from datasets import load_from_disk
load_dataset = load_from_disk("intdataset")
len(load_dataset)


Loading dataset from disk:   0%|          | 0/51 [00:00<?, ?it/s]

11373074

In [64]:
from tqdm import tqdm

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

schema = pa.schema([
    ("target", pa.int32()),
    ("context", pa.int32()),
])

SHARD_ROWS = 500_000_000 
shard_id = 0
rows = 0
buf_t, buf_c = [], []

def new_writer(i):
    return pq.ParquetWriter(
        f"shards/shard-{i:05d}.parquet",
        schema,
        compression="zstd"
    )

writer = new_writer(shard_id)

for row in tqdm(load_dataset, desc="проход по строкам"):
    toks = row["tokens"]
    for i in range(2, len(toks) - 2):
        t = toks[i]
        for c in toks[i-2:i] + toks[i+1:i+3]:
            buf_t.append(t)
            buf_c.append(c)
            rows += 1

            if rows >= SHARD_ROWS:
                table = pa.Table.from_arrays([buf_t, buf_c], schema=schema)
                writer.write_table(table)
                writer.close()

                shard_id += 1
                writer = new_writer(shard_id)
                rows = 0
                buf_t, buf_c = [], []

if buf_t:
    writer.write_table(pa.Table.from_arrays([buf_t, buf_c], schema=schema))
    writer.close()


проход по строкам:   0%|          | 3760/11373074 [00:03<2:47:38, 1130.30it/s]


KeyboardInterrupt: 

In [70]:
from multiprocessing import cpu_count

In [71]:
ds = Dataset.from_generator(gen, num_proc=cpu_count() - 1)

Setting num_proc from 11 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

KeyboardInterrupt: 

In [54]:
ds[2]

{'target': 14308, 'context': 31850}

In [52]:
load_dataset[1]

{'tokens': [67,
  57977,
  14308,
  31850,
  222631,
  127354,
  2,
  1,
  893,
  2,
  1,
  609,
  2298,
  458,
  260,
  46,
  20,
  195,
  105506,
  388929,
  361371,
  654,
  49513,
  44,
  82088,
  3306,
  1321,
  27959,
  61219,
  183,
  98828,
  4405,
  9586,
  156935,
  11180,
  4767,
  745,
  47054,
  2298,
  7389,
  39051,
  39258,
  7906,
  33660,
  347,
  8765,
  14270,
  7303,
  1194,
  508,
  26233,
  508,
  8431,
  48367,
  3013,
  111750,
  3239,
  4147,
  3116,
  152394,
  5229,
  822,
  1,
  107114,
  3983,
  7840,
  37794,
  1955,
  22917,
  259,
  8592,
  1072,
  51770,
  8592,
  21059,
  16,
  4009,
  5479,
  26702,
  1620153,
  15521,
  6014,
  20,
  12324,
  6014,
  6014,
  1,
  2507,
  4380,
  164,
  492,
  3374,
  6014,
  858,
  7389,
  3164,
  94554,
  2984,
  1218,
  39051,
  49,
  52925,
  8789,
  858,
  39051,
  7389,
  473401,
  293918,
  6565,
  97493,
  6199,
  6710,
  25,
  7906,
  7389,
  14270,
  8592,
  9821,
  33278,
  67113,
  3976,
  18,
  14270,
  